In [5]:
import math

# Assigment 1 forwad pass and backpropagation without using any library

x = [1,-1]
t = [1,0]

w = [[1,1,1],[-1,-1,-1]]
b = [0,0,0]

v = [[1,1],[-1,-1],[-1,-1]]
c = [0,0]


# forward pass

print(f"x: {x}")

# first layer
print(f"W: {w}")

k = [sum([x[i]*w[i][j] for i in range(len(x))]) + b[j] for j in range(len(b))]

print(f"k: {k}")

# sigmoid layer
z = [1/(1+math.exp(-k[i])) for i in range(len(k))]

print(f"z: {z}")

# second layer
print(f"V: {v}")

o = [sum([z[i]*v[i][j] for i in range(len(z))]) + c[j] for j in range(len(c))]

print(f"o: {o}")

# softmax layer
y = [math.exp(o[i])/sum([math.exp(o[j]) for j in range(len(o))]) for i in range(len(o))]

print(f"y: {y}")

# cross entropy loss
loss = -sum([t[i]*math.log(y[i]) for i in range(len(t))])

print(f"loss: {loss}")


x: [1, -1]
W: [[1, 1, 1], [-1, -1, -1]]
k: [2, 2, 2]
z: [0.8807970779778823, 0.8807970779778823, 0.8807970779778823]
V: [[1, 1], [-1, -1], [-1, -1]]
o: [-0.8807970779778823, -0.8807970779778823]
y: [0.5, 0.5]
loss: 0.6931471805599453


In [6]:
# backward pass
# derivative of loss function with respect to yi
dy = [(-1/y[i])*t[i] for i in range(len(y))]

print(f"dy: {dy}")

# derivative of loss function with respect to oi
do = [sum([dy[j] * (y[j]*(1-y[j]) if j==i else -y[j]*y[i]) for j in range(len(dy))]) for i in range(len(o))]

print(f"do: {do}")

# derivative of loss function with respect to v
dv = [[do[j]*z[i] for j in range(len(do))] for i in range(len(z))]

print(f"dv: {dv}")

# derivative of loss function with respect to ci
dc = [do[i]  for i in range(len(c))]

print(f"dc: {dc}")

# derivative of loss function with respect to zi
dz = [sum([do[j]*v[i][j] for j in range(len(do))]) for i in range(len(v))]

print(f"dz: {dz}")

# derivative of loss function with respect to ki
dk = [dz[i]*z[i]*(1-z[i]) for i in range(len(z))]

print(f"dk: {dk}")

# derivative of loss function with respect to wi
dw = [[dk[j]*x[i] for j in range(len(dk))] for i in range(len(x))]

print(f"dw: {dw}")

# derivative of loss function with respect to bi
db = [dk[i] for i in range(len(b))]

print(f"db: {db}")

dy: [-2.0, -0.0]
do: [-0.5, 0.5]
dv: [[-0.44039853898894116, 0.44039853898894116], [-0.44039853898894116, 0.44039853898894116], [-0.44039853898894116, 0.44039853898894116]]
dc: [-0.5, 0.5]
dz: [0.0, 0.0, 0.0]
dk: [0.0, 0.0, 0.0]
dw: [[0.0, 0.0, 0.0], [-0.0, -0.0, -0.0]]
db: [0.0, 0.0, 0.0]


In [10]:
import math

class SimpleNeuralNetwork:
    def __init__(self, w, b, v, c):
        # Initialize weights and biases
        self.w = w
        self.b = b
        self.v = v
        self.c = c
    

    def sigmoid(self, x):
        return 1 / (1 + math.exp(-x))

    def forward_pass(self, x):
        # First layer
        k = [sum(x[i] * self.w[i][j] for i in range(len(x))) + self.b[j] for j in range(len(self.b))]
        # Sigmoid layer
        z = [self.sigmoid(k[i]) for i in range(len(k))]
        # Second layer
        o = [sum(z[i] * self.v[i][j] for i in range(len(z))) + self.c[j] for j in range(len(self.c))]
        # Softmax layer
        y = [math.exp(o[i]) / sum(math.exp(o[j]) for j in range(len(o))) for i in range(len(o))]
        return y, z, k, o

    def compute_loss(self, y, t):
        return -sum(t[i] * math.log(y[i]) for i in range(len(t)))

    def backward_pass(self,t, y, o, z, x):
        # Derivatives of the loss function
        # derivative of loss function with respect to yi
        dy = [(-t[i] / y[i]) for i in range(len(y))]

        # derivative of loss function with respect to oi
        do = [sum([dy[j] * (y[j]*(1-y[j]) if j==i else -y[j]*y[i]) for j in range(len(dy))]) for i in range(len(o))]

        # derivative of loss function with respect to v
        dv = [[do[j]*z[i] for j in range(len(do))] for i in range(len(z))]

        # derivative of loss function with respect to ci
        dc = [do[i]  for i in range(len(c))]

        # derivative of loss function with respect to zi
        dz = [sum([do[j]*self.v[i][j] for j in range(len(do))]) for i in range(len(self.v))]

        # derivative of loss function with respect to ki
        dk = [dz[i]*z[i]*(1-z[i]) for i in range(len(z))]

        # derivative of loss function with respect to wi
        dw = [[dk[j]*x[i] for j in range(len(dk))] for i in range(len(x))]

        # derivative of loss function with respect to bi
        db = [dk[i] for i in range(len(b))]

        return dw, db, dc, dv
    def update_weights(self, dw, db,dc, dv, learning_rate=0.1):
        # Update the weights and biases using the learning rate
        for i in range(len(self.w)):
            for j in range(len(self.w[i])):
                self.w[i][j] -= learning_rate * dw[i][j]
        
        for i in range(len(self.b)):
            self.b[i] -= learning_rate * db[i]

        for i in range(len(self.v)):
            for j in range(len(self.v[i])):
                self.v[i][j] -= learning_rate * dv[i][j]

        for i in range(len(self.c)):
            self.c[i] -= learning_rate * dc[i]

    def train(self, xtrain, ytrain, epochs):
        for epoch in range(epochs):
            total_loss = 0
            for index,(x, t) in enumerate(zip(xtrain, ytrain)):
                y, z, k, o = self.forward_pass(x)
                loss = self.compute_loss(y, t)
                dw, db, dc, dv = self.backward_pass(t, y, o, z, x)
                # if is first run print derivative
                if epoch == 0 and index == 0:
                    print(f"dw: {dw}")
                    print(f"db: {db}")
                    print(f"dc: {dc}")
                    print(f"dv: {dv}")
                self.update_weights(dw, db, dc, dv)
                total_loss += loss
            avg_loss = total_loss / len(xtrain)
            print(f"Epoch {epoch + 1}, Average Loss: {avg_loss}")
            
    def validate(self, xval, yval):
        total_loss = 0
        correct_predictions = 0
        for x, t in zip(xval, yval):
            y, _, _, _ = self.forward_pass(x)

            # Predicted class is the one with the highest probability
            predicted_class = y.index(max(y))

            # Actual class is the one with the highest value in the target vector
            actual_class = t.index(max(t))

            # Increment correct predictions if the classes match
            if predicted_class == actual_class:
                correct_predictions += 1

            loss = self.compute_loss(y, t)
            total_loss += loss

        avg_loss = total_loss / len(xval)
        accuracy = correct_predictions / len(xval)
        return avg_loss, accuracy

# Example usage
x = [[1, -1], [-1, 1]]
t = [[1, 0], [0, 1]]
w = [[1, 1, 1], [-1, -1, -1]]
b = [0, 0, 0]
v = [[1, 1], [-1, -1], [-1, -1]]
c = [0, 0]
nn = SimpleNeuralNetwork(w, b, v, c)
nn.train(x, t, 100)



dw: [[0.0, 0.0, 0.0], [-0.0, -0.0, -0.0]]
db: [0.0, 0.0, 0.0]
dc: [-0.5, 0.5]
dv: [[-0.44039853898894116, 0.44039853898894116], [-0.44039853898894116, 0.44039853898894116], [-0.44039853898894116, 0.44039853898894116]]
Epoch 1, Average Loss: 0.7271016556975978
Epoch 2, Average Loss: 0.6833104722571401
Epoch 3, Average Loss: 0.6478012119807957
Epoch 4, Average Loss: 0.6179929318115058
Epoch 5, Average Loss: 0.5921725289488446
Epoch 6, Average Loss: 0.5692155385385275
Epoch 7, Average Loss: 0.5483821001909135
Epoch 8, Average Loss: 0.5291803634833332
Epoch 9, Average Loss: 0.51127833919219
Epoch 10, Average Loss: 0.49444772247397856
Epoch 11, Average Loss: 0.4785281028622107
Epoch 12, Average Loss: 0.4634040280354398
Epoch 13, Average Loss: 0.4489901738010207
Epoch 14, Average Loss: 0.43522165760908116
Epoch 15, Average Loss: 0.4220476452513322
Epoch 16, Average Loss: 0.40942708780831905
Epoch 17, Average Loss: 0.39732585128852094
Epoch 18, Average Loss: 0.38571476632982704
Epoch 19, Aver

In [9]:
# print weights and biases of the network
print(f"w: {nn.w}")
print(f"b: {nn.b}")
print(f"v: {nn.v}")
print(f"c: {nn.c}")

w: [[1.3190182013711846, 1.3190182013711844, 1.3190182013711844], [-1.3190182013711846, -1.3190182013711844, -1.3190182013711844]]
b: [-0.10298842004189203, -0.102988420041892, -0.102988420041892]
v: [[1.9765982502944903, 0.02340174970551011], [-0.023401749705510153, -1.9765982502944905], [-0.023401749705510153, -1.9765982502944905]]
c: [-1.2382454716823024, 1.2382454716823015]
